In [1]:
import os
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from keras.models import Model
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import BatchNormalization
from keras.utils import np_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
train_labels = []

xml = ['AA_Crossing_01-D.xml', 'AA_Easy_01-D.xml', 'AA_Easy_02-D.xml', 'AA_Easy_Entrance-D.xml', 'Munich01-D.xml', 'AA_Walking_01-D.xml', 'RaR_Snack_Zone_01-D.xml']

for file_name in xml:
    print(file_name)
    tree = ET.parse("./Data_Sets/IPF_Detections/" + file_name)
    root = tree.getroot()

    for frame in root:
        file = frame.attrib['file'].split('/')
        file = "./Data_Sets/x_train/" + file[-1]
        for objectlist in frame:
            for object in objectlist:
                label = []
                label.append(file)
                label.append(object[0].attrib['xc'])
                label.append(object[0].attrib['yc'])
                label.append(object[0].attrib['w'])
                label.append(object[0].attrib['h'])
                train_labels.append(label)


AA_Crossing_01-D.xml
AA_Easy_01-D.xml
AA_Easy_02-D.xml
AA_Easy_Entrance-D.xml
Munich01-D.xml
AA_Walking_01-D.xml
RaR_Snack_Zone_01-D.xml


In [5]:
test_labels = []

xml = ['AA_Crossing_02-D.xml', 'AA_Entrance_01-D.xml', 'AA_Walking_02-D.xml', 'AA_Easy_Entrance-D.xml', 'Munich02-D.xml', 'RaR_Snack_Zone_02-D.xml', 'RaR_Snack_Zone_04-D.xml']

for file_name in xml:
    print(file_name)
    tree = ET.parse("./Data_Sets/IPF_Detections/" + file_name)
    root = tree.getroot()

    for frame in root:
        file = frame.attrib['file'].split('/')
        file = "./Data_Sets/x_test/" + file[-1]
        for objectlist in frame:
            for object in objectlist:
                label = []
                label.append(file)
                label.append(object[0].attrib['xc'])
                label.append(object[0].attrib['yc'])
                label.append(object[0].attrib['w'])
                label.append(object[0].attrib['h'])
                test_labels.append(label)

AA_Crossing_02-D.xml
AA_Entrance_01-D.xml
AA_Walking_02-D.xml
AA_Easy_Entrance-D.xml
Munich02-D.xml
RaR_Snack_Zone_02-D.xml
RaR_Snack_Zone_04-D.xml


In [6]:
area = (0, 0, 409, 387)
x_train = []
y_train = []
# i = 1
for file in train_labels:
    filename = file[0].split('/')[-1]
    if filename.endswith(".png"):
        try:
            img = load_img("./Data_Sets/x_train/" + filename)
        except:
            continue
        cropped_img = img.crop(area)
        img_array = img_to_array(cropped_img) / 255
        x_train.append(img_array)
        y_train.append(file[1:])
    else:
        continue

In [ ]:
area = (0, 0, 409, 387)
x_test = []
y_test = []
# i = 1
for file in test_labels:
    filename = file[0].split('/')[-1]
    if filename.endswith(".png"):
        try:
            img = load_img("./Data_Sets/x_test/" + filename)
        except:
            continue
        cropped_img = img.crop(area)
        img_array = img_to_array(cropped_img) / 255
        x_test.append(img_array)
        y_test.append(file[1:])
    else:
        continue

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [2]:
# # Create model
# model = Sequential()

# # Add first convolutional layer - output:(32x32x32)
# model.add(Conv2D(input_shape=(387, 409, 3), data_format='channels_last',
#                 filters=20, kernel_size=(5,5), strides=(1,1),
#                 padding='same', kernel_initializer='he_uniform'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# # Add max-pooling layer (16x16x32)
# model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same',
#                        data_format='channels_last'))

# # Add second convolutional layer (14x14x64)
# model.add(Conv2D(filters=20, kernel_size=(5,5), strides=(1,1),
#                  padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# # Add max-pooling layer (6x6x128)
# model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same',
#                        data_format='channels_last'))

# # Add third convolutional layer (12x12x128)
# model.add(Conv2D(filters=20, kernel_size=(5,5), strides=(1,1),
#                  padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# # Add max-pooling layer (6x6x128)
# model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same',
#                        data_format='channels_last'))


# #Add up-sampling layer
# model.add(UpSampling2D(size=(2, 2), data_format=None))

# # Add third convolutional layer (12x12x128)
# model.add(Conv2D(filters=20, kernel_size=(5,5), strides=(1,1),
#                  padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# #Add up-sampling layer
# model.add(UpSampling2D(size=(2, 2), data_format=None))


# # Add second convolutional layer (14x14x64)
# model.add(Conv2D(filters=20, kernel_size=(5,5), strides=(1,1),
#                  padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# #Add up-sampling layer
# model.add(UpSampling2D(size=(2, 2), data_format=None))

# # Add second convolutional layer (14x14x64)
# model.add(Conv2D(filters=20, kernel_size=(5,5), strides=(1,1),
#                  padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# # Add second convolutional layer (14x14x64)
# model.add(Conv2D(filters=3, kernel_size=(8,1), strides=(1,1),
#                  padding='valid'))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.2))

# model.summary()

# model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(387,409,3))

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(4, activation='linear')(x) #minor edit
new_model = Model(input=model.input, output=x)

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
new_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
#fit model
new_model.fit(x_train, y_train, epochs=25, batch_size=128,
          shuffle=True)

In [ ]:
new_model.summary()

In [ ]:
#[NEEDED] method to draw box around image given prediction coords
bbox = model.predict(x_test)
array_to_img(reconstructed_imgs(bbox) * 255)